# Code is released under Apache 2.0 License

In [1]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!pip install --no-index /kaggle/input/dim-dependencies/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/imc2024-official-weights/* /root/.cache/torch/hub/checkpoints

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
kornia-rs is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Processing /kaggle/input/dim-depen

In [2]:
import gc
import os
import cv2
import sys
import torch
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import multiprocessing
import kornia.feature as KF
from torchmetrics import StructuralSimilarityIndexMeasure
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

sys.path.append('/kaggle/input')
from imc24lightglue import ALIKED
from imc24lightglue.utils import read_image, numpy_image_to_torch

INPUT_ROOT = '/kaggle/input/image-matching-challenge-2024'
OUTPUT_ROOT = '/kaggle/working'

DEBUG = False

In [3]:
if DEBUG:
    scenes = ["transp_obj_glass_cylinder"]
    categories_df = pd.read_csv("/kaggle/input/image-matching-challenge-2024/train/categories.csv")
else:
    scenes = [x for x in os.listdir(f"{INPUT_ROOT}/test/") if os.path.isdir(f"{INPUT_ROOT}/test/{x}")]
    categories_df = pd.read_csv("/kaggle/input/image-matching-challenge-2024/test/categories.csv")

In [4]:
image_sizes = [1024, 1280, 1600]

aliked_extractor = ALIKED(weights="/kaggle/input/imc24lightglue/weights/aliked-n16.pth").cuda().eval()

lightglue_matcher_params = {
    "filter_threshold": 0.2,
    "width_confidence": -1,
    "depth_confidence": -1,
    "mp": True
}
lightglue_matcher = KF.LightGlueMatcher(feature_name="aliked", params=lightglue_matcher_params).cuda().eval()

ssim_cuda = StructuralSimilarityIndexMeasure(data_range=255.).cuda()

Loaded LightGlue model


/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `StructuralSimilarityIndexMeasure` from `torchmetrics` was deprecated and will be removed in 2.0. Import `StructuralSimilarityIndexMeasure` from `torchmetrics.image` instead.
  _future_warning(


In [5]:
def get_rmats(n):
    ux, uy, uz = 0, 0, 1

    thetas = [(360 / n) * i for i in range(n)]
    Rmats = []
    for theta in thetas:
        costheta = np.cos(np.deg2rad(theta)).round(8)
        sintheta = np.sin(np.deg2rad(theta)).round(8)

        r00 = costheta + ux**2*(1-costheta)
        r01 = ux * uy * (1-costheta) - uz*sintheta
        r02 = ux * uz * (1-costheta) + uy*sintheta

        r10 = uy * ux * (1-costheta) + uz*sintheta
        r11 = costheta + uy**2*(1-costheta)
        r12 = uy * uz * (1-costheta) - ux*sintheta

        r20 = uz * ux * (1-costheta) - uy*sintheta
        r21 = uz * uy * (1-costheta) + ux*sintheta
        r22 = costheta + uz**2*(1-costheta)

        Rmat = np.array([[r00, r01, r02], [r10, r11, r12], [r20, r21, r22]])
        Rmats.append(Rmat)

    return Rmats


def get_distance_matrix(fnames, flows):
    distance_matrix = np.zeros((len(fnames), len(fnames)), dtype=np.int32)
    for idx1, fname1 in enumerate(fnames):
        for idx2, fname2 in enumerate(fnames):
            if idx1 == idx2:
                value = 0
            else:
                value = flows[(fname1, fname2)]
            distance_matrix[idx1, idx2] = value
    return distance_matrix


def get_distance_matrix_2(fnames, order_idxs_list):

    neighbors_dict = {idx: [] for idx in range(len(fnames))}
    for order_idxs in order_idxs_list:
        for idx in range(len(fnames)):
            neighbor_left = order_idxs[idx-1]
            neighbor_right = order_idxs[0] if idx == len(fnames) - 1 else order_idxs[idx+1]
            neighbors_dict[order_idxs[idx]].extend([neighbor_left, neighbor_right])

    flows = {}
    for i in range(len(fnames)):
        for j in range(len(fnames)):
            if i == j:
                value = 0
            else:
                cnt = neighbors_dict[i].count(j)
                if cnt == 0:
                    value = 100000000
                else:
                    value = 1000000 // cnt
            flows[(i, j)] = value

    distance_matrix = np.zeros((len(fnames), len(fnames)), dtype=np.int32)
    for idx1 in range(len(fnames)):
        for idx2 in range(len(fnames)):
            distance_matrix[idx1, idx2] = flows[(idx1, idx2)]

    dummy_distance_matrix = np.zeros((len(fnames)+1, len(fnames)+1), dtype=np.int32)
    dummy_distance_matrix[1:,1:] = distance_matrix
    return dummy_distance_matrix


def tsp_distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]


def get_tsp_solution(manager, routing, solution):
    """Prints solution on console."""
    index = routing.Start(0)
    idxs = []
    while not routing.IsEnd(index):
        idxs.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    return np.array(idxs)


def get_img_pairs_all(fnames):
    index_pairs = []
    for i in range(len(fnames)):
        for j in range(i+1, len(fnames)):
            index_pairs.append((i,j))
    return index_pairs


def resize(image, image_size):
    h, w = image.shape[:2]
    aspect_ratio = h/w
    smaller_side_size = int(image_size/max(aspect_ratio, 1/aspect_ratio))
    if aspect_ratio > 1: # H > W
        new_size = (image_size, smaller_side_size)
    else: # H <= W
        new_size = (smaller_side_size, image_size)
    image = cv2.resize(image, new_size[::-1])
    return image, new_size


def alg_inference(cache, fname1, fname2, image_size, rot_code_2):

    # Extraction
    if 'keypoints_alg' not in cache[fname1][image_size][0]:
        with torch.inference_mode():
            tensor = cache[fname1][image_size]['tensor_alg'].cuda()
            pred = aliked_extractor.extract(tensor, resize=None)
            cache[fname1][image_size][0] = {
                **cache[fname1][image_size][0],
                **{
                    'keypoints_alg': pred['keypoints'],
                    'descriptors_alg': pred['descriptors']
                }
            }

    if 'keypoints_alg' not in cache[fname2][image_size][rot_code_2]:
        with torch.inference_mode():
            tensor = torch.rot90(cache[fname2][image_size]['tensor_alg'], rot_code_2, [1, 2]).cuda()
            pred = aliked_extractor.extract(tensor, resize=None)
            cache[fname2][image_size][rot_code_2] = {
                **cache[fname2][image_size][rot_code_2],
                **{
                    'keypoints_alg': pred['keypoints'],
                    'descriptors_alg': pred['descriptors']
                }
            }

    # Matching
    kpts1, kpts2 = cache[fname1][image_size][0]['keypoints_alg'], cache[fname2][image_size][rot_code_2]['keypoints_alg']
    with torch.inference_mode():
        _, indices = lightglue_matcher(
            cache[fname1][image_size][0]['descriptors_alg'][0], 
            cache[fname2][image_size][rot_code_2]['descriptors_alg'][0],
            KF.laf_from_center_scale_ori(kpts1),
            KF.laf_from_center_scale_ori(kpts2)
        )
        kpts1 = kpts1[0].cpu().numpy()
        kpts2 = kpts2[0].cpu().numpy()
        indices = indices.cpu().numpy()
        
    mkpts1 = kpts1[indices[..., 0]].astype(np.float32)
    mkpts2 = kpts2[indices[..., 1]].astype(np.float32)

    try:
        _, inliers = cv2.findFundamentalMat(mkpts1, mkpts2, cv2.USAC_MAGSAC, ransacReprojThreshold=5, confidence=0.9999, maxIters=50000)
        inliers = inliers.ravel() > 0
        mkpts1 = mkpts1[inliers]
        mkpts2 = mkpts2[inliers]
    except:
        pass

    num_matches = len(mkpts1)

    return num_matches


def matching_inference(fname1, fname2, cache=None):

    for fname in [fname1, fname2]:
        if fname not in cache:
            img = read_image(fname)
            h, w = img.shape[:2]
            cache[fname] = {'image': img, 'h': h, 'w': w} 
            for image_size in image_sizes:
                if max(h, w) != image_size:
                    img_r, (h_r, w_r) = resize(img, image_size)
                else:
                    img_r = img.copy()
                    h_r, w_r = img_r.shape[:2]
                tensor_alg = numpy_image_to_torch(img_r)
                cache[fname][image_size] = {'tensor_alg': tensor_alg, 'h_r': h_r, 'w_r': w_r, 0: {}, 1: {}, 2: {}, 3: {}}

    # Co-orientation
    rot_code_2, max_n = 0, 0
    for rc2 in range(4):
        n = alg_inference(cache, fname1, fname2, image_sizes[0], rot_code_2=rc2)
        if n > max_n:
            rot_code_2 = rc2
            max_n = n

    num_matches = max_n
    for image_size in image_sizes[1:]:
        num_matches += alg_inference(cache, fname1, fname2, image_size, rot_code_2)

    return num_matches


def get_matching_flows(fnames):
    index_pairs = get_img_pairs_all(fnames=fnames)
    cache, flows = {}, {}
    for pair_idx in tqdm(index_pairs, desc="Matching"):
        idx1, idx2 = pair_idx
        fname1, fname2 = fnames[idx1], fnames[idx2]
        num_matches = matching_inference(fname1, fname2, cache)
        flows[(fname1, fname2)] = flows[(fname2, fname1)] = int((1 / num_matches) * 1e8)
    return flows


def compute_ssim(im1, im2):
    with torch.inference_mode():
        tensor1 = torch.tensor(im1.copy(), dtype=torch.float32)[None][None].cuda()
        tensor2 = torch.tensor(im2.copy(), dtype=torch.float32)[None][None].cuda()
        score = ssim_cuda(tensor1, tensor2).cpu().numpy().item()
    return score


def get_flows(fnames):
    relative_orientations = {fnames[0]: 0}
    flow_images = [cv2.imread(fname, cv2.IMREAD_GRAYSCALE) for fname in fnames]
    flow_images = [resize(image, 2048)[0] for image in flow_images]
    ssim_flows = {}
    for i in tqdm(range(len(fnames)), desc="Getting flows"):
        image_i = flow_images[i]
        for j in range(i+1, len(fnames)):
            image_j = flow_images[j]
            if i == 0:
                diffs = []
                try:
                    diffs.append(compute_ssim(image_i, image_j))
                except:
                    diffs.append(0)
                for rot_code in range(1,4):
                    try:
                        diffs.append(compute_ssim(image_i, np.rot90(image_j, rot_code)))
                    except:
                        diffs.append(0)
                min_diff_idx = np.argmax(diffs)
                relative_orientations[fnames[j]] = min_diff_idx
            
            r1, r2 = relative_orientations[fnames[i]], relative_orientations[fnames[j]]
            im1 = np.rot90(image_i, r1) if r1 else image_i
            im2 = np.rot90(image_j, r2) if r2 else image_j

            ssim_score = 1 - compute_ssim(im1, im2)
            ssim_flows[(fnames[i], fnames[j])] = ssim_flows[(fnames[j], fnames[i])] = ssim_score

    return ssim_flows


def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])

In [6]:
results_df = pd.DataFrame(columns=['image_path', 'dataset', 'scene', 'rotation_matrix', 'translation_vector'])
non_transparent_scenes = []
for scene in tqdm(scenes, desc='Running pipeline'):

    torch.cuda.empty_cache()
    gc.collect()
    
    categories = categories_df[categories_df["scene"]==scene]["categories"].item()
    use_crops = ("symmetr" not in categories) and ("transparen" not in categories)
    is_transparent = ("transparen" in categories)

    print(f"{scene=} {categories=} {use_crops=} {is_transparent=}")

    img_dir = f"{INPUT_ROOT}/{'train' if DEBUG else 'test'}/{scene}/images"
    if not os.path.exists(img_dir):
        continue

    fnames = sorted(glob(f"{img_dir}/*"))
    
    if is_transparent:
        Rmats = get_rmats(len(fnames))
    
        ssim_flows = get_flows(fnames)
        matching_flows = get_matching_flows(fnames)

        ssim_min, ssim_max = min(list(ssim_flows.values())), max(list(ssim_flows.values()))
        matching_min, matching_max = min(list(matching_flows.values())), max(list(matching_flows.values()))

        ssim_flows = {k: (v - ssim_min) / (ssim_max - ssim_min) for k, v in ssim_flows.items()}
        matching_flows = {k: (v - matching_min) / (matching_max - matching_min) for k, v in matching_flows.items()}

        flows = {k:int((ssim_flows[k] + matching_flows[k]) * 1e6) for k in ssim_flows.keys()}
        
        distance_matrix = get_distance_matrix(fnames, flows)
        
        ############# N + 1 #############
        order_idxs_list = []
        for start_idx in range(len(fnames)):
            manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, start_idx)
            routing = pywrapcp.RoutingModel(manager)
            transit_callback_index = routing.RegisterTransitCallback(tsp_distance_callback)
            routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
            search_parameters = pywrapcp.DefaultRoutingSearchParameters()
            search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
            solution = routing.SolveWithParameters(search_parameters)
            order_idxs = get_tsp_solution(manager, routing, solution)
            order_idxs_list.append(order_idxs)

        distance_matrix = get_distance_matrix_2(fnames, order_idxs_list)

        manager = pywrapcp.RoutingIndexManager(len(distance_matrix), 1, 0)
        routing = pywrapcp.RoutingModel(manager)
        transit_callback_index = routing.RegisterTransitCallback(tsp_distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        solution = routing.SolveWithParameters(search_parameters)
        order_idxs = get_tsp_solution(manager, routing, solution)
        order_idxs = order_idxs[1:] - 1
        ############# N + 1 #############
    else:
        non_transparent_scenes.append(scene)
        continue

    # Create submission
    for fid, fname in enumerate(fnames):
        image_id = '/'.join(fname.split('/')[-4:])
        if is_transparent:
            R = Rmats[np.where(order_idxs==fid)[0].item()]
            T = np.array([100, 100, 100])
        elif image_id in results:
            R = results[image_id]['R']
            T = results[image_id]['t']
        else:
            R = np.eye(3)
            T = np.zeros(3)

        new_row = pd.DataFrame({'image_path': image_id,
                                'dataset': scene,
                                'scene': scene,
                                'rotation_matrix': arr_to_str(R),
                                'translation_vector': arr_to_str(T)}, index=[0])

        results_df = pd.concat([results_df, new_row]).reset_index(drop=True)

Running pipeline: 100%|██████████| 1/1 [00:00<00:00,  5.38it/s]

scene='church' categories='symmetries-and-repeats' use_crops=False is_transparent=False


In [7]:
results_df.to_csv(f"{OUTPUT_ROOT}/transp_submission.csv", index=False)

In [8]:
results_df.head()

,image_path,dataset,scene,rotation_matrix,translation_vector


In [9]:
import json
with open("/kaggle/working/non_transparent_scenes.json", "w") as f:
    json.dump(non_transparent_scenes, f)

In [10]:
aliked_extractor.cpu()
lightglue_matcher.cpu()
ssim_cuda.cpu()

del aliked_extractor, lightglue_matcher, ssim_cuda
torch.cuda.empty_cache()
gc.collect()

72

---

# No Transparent Part

## Prepare Inference Scripts

In [11]:
!cp -r /kaggle/input/imc2024-inference-scripts /kaggle/working/

In [12]:
%%writefile imc2024-inference-scripts/config.py

import os
import json

class Config:
    input_dir_root = "/kaggle/input"
    output_dir = "/tmp"
    check_exist_dir = False
    
    input_csv = "/kaggle/input/image-matching-challenge-2024/sample_submission.csv"
    category_csv = "/kaggle/input/image-matching-challenge-2024/test/categories.csv"
    target_datasets = []

    pipeline_json = "/kaggle/input/configs-imc2024/1280_2048_crop_1280_2048/pipeline.json"
    transparent_pipeline_json = "/kaggle/input/imc2024-pipelines/lg1280+1536+1024+2048_crop-lg1024+1280+1536/transp_pipeline.json"
    
    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 5,
        #"num_threads": 1,
    }

Overwriting imc2024-inference-scripts/config.py


In [13]:
%%writefile imc2024-inference-scripts/pipeline.py

import os
import time
import pandas as pd
import numpy as np
import gc
import kornia as K

from tasks.get_pair.get_image_pair_exhaustive import task_get_image_pair_exhaustive
from tasks.get_pair.get_image_pair_DINO import task_get_image_pair_DINO
from tasks.get_pair.get_image_pair_kNN import task_get_image_pair_kNN
from tasks.get_pair.get_transparent_pair import task_get_transparent_pair
from tasks.matching.matching import task_matching
from tasks.matching.rotate_matching_find_best import task_rotate_matching_find_best
from tasks.crop.sfm_mkpc import task_sfm_mkpc
from tasks.crop.pair_mkpc import task_pair_mkpc
from tasks.crop.transparent_crop import task_transparent_crop
from tasks.utils.ransac import task_ransac
from tasks.utils.concat import task_concat
from tasks.utils.rem_less_match_pair import task_rem_less_match_pair
from tasks.utils.count_matching_num import task_count_matching_num
from tasks.utils.extract_inliner_matching_points import task_extract_inliner_matching_points
from tasks.utils.extract_csv_pair import task_extract_csv_pair
from tasks.utils.estimate_rot import task_estimate_rot
from tasks.utils.get_exif import task_get_exif

task_map = {
    "get_image_pair_exhaustive": task_get_image_pair_exhaustive,
    "get_image_pair_DINO": task_get_image_pair_DINO,
    "get_image_pair_kNN": task_get_image_pair_kNN,
    "get_transparent_pair": task_get_transparent_pair,

    "matching": task_matching,
    "rotate_matching_find_best": task_rotate_matching_find_best,

    "sfm_mkpc": task_sfm_mkpc,
    "pair_mkpc": task_pair_mkpc,
    "transparent_crop": task_transparent_crop,
    
    "ransac": task_ransac,
    "concat": task_concat,
    "rem_less_match_pair": task_rem_less_match_pair,
    "count_matching_num": task_count_matching_num,
    "extract_inliner_matching_points": task_extract_inliner_matching_points,
    "extract_csv_pair": task_extract_csv_pair,
    "estimate_rot": task_estimate_rot,
    "get_exif": task_get_exif,
}

class Pipeline():
    def __init__(self, data_dict, work_dir, input_dir_root, pipeline_config, device_id):
        self.device = K.utils.get_cuda_device_if_available(device_id)
        print(f"device: {self.device}")
        self.data_dict = data_dict
        self.work_dir = work_dir
        self.input_dir_root = input_dir_root
        self.pipeline_config = pipeline_config
        self.processing_times = {
            "task": [],
            "comment": [],
            "processing_time": []
        }

    def exec(self):
        all_processing_time = 0
        for p in self.pipeline_config:
            task = p["task"]
            comment = p["comment"]
            p["params"]["device"] = self.device
            p["params"]["data_dict"] = self.data_dict
            p["params"]["work_dir"] = self.work_dir
            p["params"]["input_dir_root"] = self.input_dir_root
            if "pdb" in p and p["pdb"]:
                p["params"]["pdb"] = True
            else:
                p["params"]["pdb"] = False
            
            start = time.time()
            print(f"===== [{task}] {comment} =====")
            task_map[task](p["params"])
            gc.collect()
            print("====================")
            end = time.time()

            self.processing_times["task"].append(task)
            self.processing_times["comment"].append(comment)
            self.processing_times["processing_time"].append(end-start)
            all_processing_time += end-start
        
        self.processing_times["task"].append("All")
        self.processing_times["comment"].append("")
        self.processing_times["processing_time"].append(all_processing_time)
        processing_times_df = pd.DataFrame.from_dict(self.processing_times)
        processing_times_df.to_csv(os.path.join(self.work_dir, "processing_time.csv"), index=False)

Overwriting imc2024-inference-scripts/pipeline.py


In [14]:
%%writefile imc2024-inference-scripts/reconstruction.py

import os
import numpy as np
from pathlib import Path
from copy import deepcopy
import pycolmap
import pandas as pd

import sqlite3
from PIL import Image, ExifTags
import h5py
from tqdm import tqdm
import warnings
import pickle
import json

# import sys
# sys.path.append("/mnt/2ndHDD/kaggle/IMC2024/datas/input/colmap-db-import")
# from database import *
# from h5_to_db import *

# def import_into_colmap(
#     path: Path,
#     feature_dir: Path,
#     database_path: str = "colmap.db",
# ) -> None:
#     """Adds keypoints into colmap"""
#     db = COLMAPDatabase.connect(database_path)
#     db.create_tables()
#     single_camera = False
#     fname_to_id = add_keypoints(db, feature_dir, path, "", "simple-pinhole", single_camera)
#     add_matches(
#         db,
#         feature_dir,
#         fname_to_id,
#     )
#     db.commit()

def arr_to_str(a):
    """Returns ;-separated string representing the input"""
    return ";".join([str(x) for x in a.reshape(-1)])

MAX_IMAGE_ID = 2**31 - 1


CREATE_CAMERAS_TABLE = """CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL)"""


CREATE_DESCRIPTORS_TABLE = """CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)"""


CREATE_IMAGES_TABLE = """CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id))
""".format(MAX_IMAGE_ID)


CREATE_TWO_VIEW_GEOMETRIES_TABLE = """
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB)
"""


CREATE_KEYPOINTS_TABLE = """CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE)
"""


CREATE_MATCHES_TABLE = """CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB)"""


CREATE_NAME_INDEX = \
    "CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)"


CREATE_ALL = "; ".join([
    CREATE_CAMERAS_TABLE,
    CREATE_IMAGES_TABLE,
    CREATE_KEYPOINTS_TABLE,
    CREATE_DESCRIPTORS_TABLE,
    CREATE_MATCHES_TABLE,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE,
    CREATE_NAME_INDEX
])


def image_ids_to_pair_id(image_id1, image_id2):
    if image_id1 > image_id2:
        image_id1, image_id2 = image_id2, image_id1
    return image_id1 * MAX_IMAGE_ID + image_id2


def array_to_blob(array):
    return array.tostring()


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path):
        return sqlite3.connect(database_path, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):
        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL)
        self.create_cameras_table = \
            lambda: self.executescript(CREATE_CAMERAS_TABLE)
        self.create_descriptors_table = \
            lambda: self.executescript(CREATE_DESCRIPTORS_TABLE)
        self.create_images_table = \
            lambda: self.executescript(CREATE_IMAGES_TABLE)
        self.create_two_view_geometries_table = \
            lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE)
        self.create_keypoints_table = \
            lambda: self.executescript(CREATE_KEYPOINTS_TABLE)
        self.create_matches_table = \
            lambda: self.executescript(CREATE_MATCHES_TABLE)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX)

    def add_camera(self, model, width, height, params,
                   prior_focal_length=0, camera_id=None):
        params = np.asarray(params, np.float64)
        cursor = self.execute(
            "INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)",
            (camera_id, model, width, height, array_to_blob(params),
             prior_focal_length))
        return cursor.lastrowid

    def add_image(self, name, camera_id,
                  prior_q=np.zeros(4), prior_t=np.zeros(3), image_id=None):
        cursor = self.execute(
            "INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
            (image_id, name, camera_id, prior_q[0], prior_q[1], prior_q[2],
             prior_q[3], prior_t[0], prior_t[1], prior_t[2]))
        return cursor.lastrowid

    def add_keypoints(self, image_id, keypoints):
        assert(len(keypoints.shape) == 2)
        assert(keypoints.shape[1] in [2, 4, 6])

        keypoints = np.asarray(keypoints, np.float32)
        self.execute(
            "INSERT INTO keypoints VALUES (?, ?, ?, ?)",
            (image_id,) + keypoints.shape + (array_to_blob(keypoints),))

    def add_matches(self, image_id1, image_id2, matches):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)
        if image_id1 > image_id2:
            matches = matches[:,::-1]
        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        self.execute(
            "INSERT INTO matches VALUES (?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches),))

    def add_two_view_geometry(self, image_id1, image_id2, matches, F=np.eye(3), E=np.eye(3), H=np.eye(3), config=2):
        assert(len(matches.shape) == 2)
        assert(matches.shape[1] == 2)
        if image_id1 > image_id2:
            matches = matches[:,::-1]
        pair_id = image_ids_to_pair_id(image_id1, image_id2)
        matches = np.asarray(matches, np.uint32)
        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)
        self.execute(
            "INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
            (pair_id,) + matches.shape + (array_to_blob(matches), config,
             array_to_blob(F), array_to_blob(E), array_to_blob(H)))

        
def get_focal(height, width, exif):
    max_size = max(height, width)
    focal_found, focal = False, None
    if exif is not None:
        focal_35mm = None
        for tag, value in exif.items():
            focal_35mm = None
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_35mm = float(value)
                break
        if focal_35mm is not None:
            focal_found = True
            focal = focal_35mm / 35. * max_size
            print(f"Focal found: {focal}")
    if focal is None:
        FOCAL_PRIOR = 1.2
        focal = FOCAL_PRIOR * max_size
    return focal_found, focal


def create_camera(db, height, width, exif, camera_model):
    focal_found, focal = get_focal(height, width, exif)
    if camera_model == 'simple-pinhole':
        model = 0 # simple pinhole
        param_arr = np.array([focal, width / 2, height / 2])
    if camera_model == 'pinhole':
        model = 1 # pinhole
        param_arr = np.array([focal, focal, width / 2, height / 2])
    elif camera_model == 'simple-radial':
        model = 2 # simple radial
        param_arr = np.array([focal, width / 2, height / 2, 0.1])
    elif camera_model == 'radial':
        model = 3 # radial
        param_arr = np.array([focal, width / 2, height / 2, 0., 0.])
    elif camera_model == 'opencv':
        model = 4 # opencv
        param_arr = np.array([focal, focal, width / 2, height / 2, 0., 0., 0., 0.])
    return db.add_camera(model, width, height, param_arr, prior_focal_length=int(focal_found))


def add_keypoints(db, feature_dir, h_w_exif, camera_model, single_camera=False):
    keypoint_f = h5py.File(os.path.join(feature_dir, 'keypoints.h5'), 'r')
    camera_id = None
    fname_to_id = {}
    for filename in tqdm(list(keypoint_f.keys())):
        keypoints = keypoint_f[filename][()]
        if camera_id is None or not single_camera:
            height = h_w_exif[filename]['h']
            width = h_w_exif[filename]['w']
            exif = h_w_exif[filename]['exif']
            camera_id = create_camera(db, height, width, exif, camera_model)
        image_id = db.add_image(filename, camera_id)
        fname_to_id[filename] = image_id
        db.add_keypoints(image_id, keypoints)
    return fname_to_id


def add_matches_and_fms(db, feature_dir, fname_to_id, fms):
    match_file = h5py.File(os.path.join(feature_dir, 'matches.h5'), 'r')
    added = set()
    for key_1 in match_file.keys():
        group = match_file[key_1]
        for key_2 in group.keys():
            id_1 = fname_to_id[key_1]
            id_2 = fname_to_id[key_2]
            pair_id = (id_1, id_2)
            if pair_id in added:
                warnings.warn(f'Pair {pair_id} ({id_1}, {id_2}) already added!')
                continue
            added.add(pair_id)
            matches = group[key_2][()]
            db.add_matches(id_1, id_2, matches)
            db.add_two_view_geometry(id_1, id_2, matches, fms[(key_1, key_2)])


def import_into_colmap(feature_dir, h_w_exif, fms):
    db = COLMAPDatabase.connect(f"{feature_dir}/colmap.db")
    db.create_tables()
    fname_to_id = add_keypoints(db, feature_dir, h_w_exif, camera_model='simple-radial', single_camera=False)
    add_matches_and_fms(db, feature_dir, fname_to_id, fms)
    db.commit()
    db.close()




import math
_EPS = np.finfo(float).eps * 4.0

def quaternion_matrix(quaternion):
    """Return homogeneous rotation matrix from quaternion."""
    q = np.array(quaternion, dtype=np.float64, copy=True)
    n = np.dot(q, q)
    if n < _EPS:
        # print("special case")
        return np.identity(4)
    q *= math.sqrt(2.0 / n)
    q = np.outer(q, q)
    return np.array(
        [
            [
                1.0 - q[2, 2] - q[3, 3],
                q[1, 2] - q[3, 0],
                q[1, 3] + q[2, 0],
                0.0,
            ],
            [
                q[1, 2] + q[3, 0],
                1.0 - q[1, 1] - q[3, 3],
                q[2, 3] - q[1, 0],
                0.0,
            ],
            [
                q[1, 3] - q[2, 0],
                q[2, 3] + q[1, 0],
                1.0 - q[1, 1] - q[2, 2],
                0.0,
            ],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )


def vector_norm(data, axis=None, out=None):
    """Return length, i.e. Euclidean norm, of ndarray along axis."""
    data = np.array(data, dtype=np.float64, copy=True)
    if out is None:
        if data.ndim == 1:
            return math.sqrt(np.dot(data, data))
        data *= data
        out = np.atleast_1d(np.sum(data, axis=axis))
        np.sqrt(out, out)
        return out
    data *= data
    np.sum(data, axis=axis, out=out)
    np.sqrt(out, out)
    return None

def affine_matrix_from_points(v0, v1, shear=False, scale=True, usesvd=True):
    """Return affine transform matrix to register two point sets.
    v0 and v1 are shape (ndims, -1) arrays of at least ndims non-homogeneous
    coordinates, where ndims is the dimensionality of the coordinate space.
    If shear is False, a similarity transformation matrix is returned.
    If also scale is False, a rigid/Euclidean traffansformation matrix
    is returned.
    By default the algorithm by Hartley and Zissermann [15] is used.
    If usesvd is True, similarity and Euclidean transformation matrices
    are calculated by minimizing the weighted sum of squared deviations
    (RMSD) according to the algorithm by Kabsch [8].
    Otherwise, and if ndims is 3, the quaternion based algorithm by Horn [9]
    is used, which is slower when using this Python implementation.
    The returned matrix performs rotation, translation and uniform scaling
    (if specified)."""

    v0 = np.array(v0, dtype=np.float64, copy=True)
    v1 = np.array(v1, dtype=np.float64, copy=True)

    ndims = v0.shape[0]
    if ndims < 2 or v0.shape[1] < ndims or v0.shape != v1.shape:
        raise ValueError("input arrays are of wrong shape or type")

    # move centroids to origin
    t0 = -np.mean(v0, axis=1)
    M0 = np.identity(ndims + 1)
    M0[:ndims, ndims] = t0
    v0 += t0.reshape(ndims, 1)
    t1 = -np.mean(v1, axis=1)
    M1 = np.identity(ndims + 1)
    M1[:ndims, ndims] = t1
    v1 += t1.reshape(ndims, 1)

    if shear:
        # Affine transformation
        A = np.concatenate((v0, v1), axis=0)
        u, s, vh = np.linalg.svd(A.T)
        vh = vh[:ndims].T
        B = vh[:ndims]
        C = vh[ndims : 2 * ndims]
        t = np.dot(C, np.linalg.pinv(B))
        t = np.concatenate((t, np.zeros((ndims, 1))), axis=1)
        M = np.vstack((t, ((0.0,) * ndims) + (1.0,)))
    elif usesvd or ndims != 3:
        # Rigid transformation via SVD of covariance matrix
        u, s, vh = np.linalg.svd(np.dot(v1, v0.T))
        # rotation matrix from SVD orthonormal bases
        R = np.dot(u, vh)
        if np.linalg.det(R) < 0.0:
            # R does not constitute right handed system
            R -= np.outer(u[:, ndims - 1], vh[ndims - 1, :] * 2.0)
            s[-1] *= -1.0
        # homogeneous transformation matrix
        M = np.identity(ndims + 1)
        M[:ndims, :ndims] = R
    else:
        # Rigid transformation matrix via quaternion
        # compute symmetric matrix N
        xx, yy, zz = np.sum(v0 * v1, axis=1)
        xy, yz, zx = np.sum(v0 * np.roll(v1, -1, axis=0), axis=1)
        xz, yx, zy = np.sum(v0 * np.roll(v1, -2, axis=0), axis=1)
        N = [
            [xx + yy + zz, 0.0, 0.0, 0.0],
            [yz - zy, xx - yy - zz, 0.0, 0.0],
            [zx - xz, xy + yx, yy - xx - zz, 0.0],
            [xy - yx, zx + xz, yz + zy, zz - xx - yy],
        ]
        # quaternion: eigenvector corresponding to most positive eigenvalue
        w, V = np.linalg.eigh(N)
        q = V[:, np.argmax(w)]
        # print (vector_norm(q), np.linalg.norm(q))
        q /= vector_norm(q)  # unit quaternion
        # homogeneous transformation matrix
        M = quaternion_matrix(q)

    if scale and not shear:
        # Affine transformation; scale is ratio of RMS deviations from centroid
        v0 *= v0
        v1 *= v1
        M[:ndims, :ndims] *= math.sqrt(np.sum(v1) / np.sum(v0))

    # move centroids back
    M = np.dot(np.linalg.inv(M1), np.dot(M, M0))
    M /= M[ndims, ndims]
    return M



def register_by_Horn(ev_coord, gt_coord, ransac_threshold, inl_cf, strict_cf):
    """Return the best similarity transforms T that registers 3D points pt_ev in <ev_coord> to
    the corresponding ones pt_gt in <gt_coord> according to a RANSAC-like approach for each
    threshold value th in <ransac_threshold>.

    Given th, each triplet of 3D correspondences is examined if not already present as strict inlier,
    a correspondence is a strict inlier if <strict_cf> * err_best < th, where err_best is the registration
    error for the best model so far.
    The minimal model given by the triplet is then refined using also its inliers if their total is greater
    than <inl_cf> * ninl_best, where ninl_best is th number of inliers for the best model so far. Inliers
    are 3D correspondences (pt_ev, pt_gt) for which the Euclidean distance |pt_gt-T*pt_ev| is less than th.
    """

    # remove invalid cameras, the index is returned
    idx_cams = np.all(np.isfinite(ev_coord), axis=0)
    ev_coord = ev_coord[:, idx_cams]
    gt_coord = gt_coord[:, idx_cams]

    # initialization
    n = ev_coord.shape[1]
    r = ransac_threshold.shape[0]
    ransac_threshold = np.expand_dims(ransac_threshold, axis=0)
    ransac_threshold2 = ransac_threshold**2
    ev_coord_1 = np.vstack((ev_coord, np.ones(n)))
    max_no_inl = np.zeros((1, r))
    best_inl_err = np.full(r, np.Inf)
    best_transf_matrix = np.zeros((r, 4, 4))
    best_err = np.full((n, r), np.Inf)
    strict_inl = np.full((n, r), False)
    triplets_used = np.zeros((3, r))

    # run on camera triplets
    for ii in range(n - 2):
        for jj in range(ii + 1, n - 1):
            for kk in range(jj + 1, n):
                i = [ii, jj, kk]
                triplets_used_now = np.full((n), False)
                triplets_used_now[i] = True
                # if both ii, jj, kk are strict inliers for the best current model just skip
                if np.all(strict_inl[i]):
                    continue
                # get transformation T by Horn on the triplet camera center correspondences
                transf_matrix = affine_matrix_from_points(
                    ev_coord[:, i], gt_coord[:, i], usesvd=False
                )
                # apply transformation T to test camera centres
                rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                # compute error and inliers
                err = np.sum((rotranslated - gt_coord) ** 2, axis=0)
                inl = np.expand_dims(err, axis=1) < ransac_threshold2
                no_inl = np.sum(inl, axis=0)
                # if the number of inliers is close to that of the best model so far, go for refinement
                to_ref = np.squeeze(
                    ((no_inl > 2) & (no_inl > max_no_inl * inl_cf)), axis=0
                )
                for q in np.argwhere(to_ref):
                    qq = q[0]
                    if np.any(
                        np.all(
                            (np.expand_dims(inl[:, qq], axis=1) == inl[:, :qq]), axis=0
                        )
                    ):
                        # already done for this set of inliers
                        continue
                    # get transformation T by Horn on the inlier camera center correspondences
                    transf_matrix = affine_matrix_from_points(
                        ev_coord[:, inl[:, qq]], gt_coord[:, inl[:, qq]]
                    )
                    # apply transformation T to test camera centres
                    rotranslated = np.matmul(transf_matrix[:3], ev_coord_1)
                    # compute error and inliers
                    err_ref = np.sum((rotranslated - gt_coord) ** 2, axis=0)
                    err_ref_sum = np.sum(err_ref, axis=0)
                    err_ref = np.expand_dims(err_ref, axis=1)
                    inl_ref = err_ref < ransac_threshold2
                    no_inl_ref = np.sum(inl_ref, axis=0)
                    # update the model if better for each threshold
                    to_update = np.squeeze(
                        (no_inl_ref > max_no_inl)
                        | ((no_inl_ref == max_no_inl) & (err_ref_sum < best_inl_err)),
                        axis=0,
                    )
                    if np.any(to_update):
                        triplets_used[0, to_update] = ii
                        triplets_used[1, to_update] = jj
                        triplets_used[2, to_update] = kk
                        max_no_inl[:, to_update] = no_inl_ref[to_update]
                        best_err[:, to_update] = np.sqrt(err_ref)
                        best_inl_err[to_update] = err_ref_sum
                        strict_inl[:, to_update] = (
                            best_err[:, to_update]
                            < strict_cf * ransac_threshold[:, to_update]
                        )
                        best_transf_matrix[to_update] = transf_matrix
    
    for i in range(r):
        print(
            f"Registered cameras {int(max_no_inl[0, i])}/{n} for threshold {ransac_threshold[0, i]}"
        )

    best_model = {
        "valid_cams": idx_cams,
        "no_inl": max_no_inl,
        "err": best_err,
        "triplets_used": triplets_used,
        "transf_matrix": best_transf_matrix,
    }    
    return best_model


def reconstruction(data_dict, dataset, scene, base_path, work_dir, colmap_mapper_options):
    # Import keypoint distances of matches into colmap for RANSAC 
    images_dir = data_dict[dataset][scene][0].parent
    with open(os.path.join(work_dir, "h_w_exif.json"), "r") as f:
        h_w_exif = json.load(f)
    
    with open(os.path.join(work_dir, "fms.pkl"), "rb") as f:
        fms = pickle.load(f)
    import_into_colmap(feature_dir=work_dir, h_w_exif=h_w_exif, fms=fms)

    database_path = f"{work_dir}/colmap.db"
    mapper_options = pycolmap.IncrementalPipelineOptions(**colmap_mapper_options)
    output_path = f"{work_dir}/colmap_rec_aliked"
    os.makedirs(output_path, exist_ok=True)

    db = COLMAPDatabase.connect(database_path)
    cursor = db.execute("SELECT image_id, name from images")
    db_data = cursor.fetchall()
    image_ids = [int(x[0]) for x in db_data]
    names = [str(x[1]) for x in db_data]
    db.close()

    df = pd.read_csv(f"{work_dir}/image_pair.csv")
    images_matches = {}
    for name in names:
        images_matches[name] = [0, 0]
    for i, row in df.iterrows():
        key1 = row["key1"]
        key2 = row["key2"]
        match_num = row["match_num"]
        images_matches[key1][0] += 1
        images_matches[key1][1] += match_num
        images_matches[key2][0] += 1
        images_matches[key2][1] += match_num
    sorted_images_matches = sorted(images_matches.items(), key=lambda item: (item[1][0], item[1][1]), reverse=True)
    init_image_name1 = sorted_images_matches[0][0]
    init_image_id1 = image_ids[names.index(init_image_name1)]
    mapper_options.init_image_id1 = init_image_id1

    maps = pycolmap.incremental_mapping(database_path=database_path, image_path=images_dir, output_path=output_path, options=mapper_options)
    print(maps)

    # 2. Look for the best reconstruction: The incremental mapping offered by 
    # pycolmap attempts to reconstruct multiple models, we must pick the best one
    images_registered  = 0
    best_idx = None
    
    print ("Looking for the best reconstruction")

    if isinstance(maps, dict):
        for idx1, rec in maps.items():
            print(idx1, rec.summary())
            try:
                if len(rec.images) > images_registered:
                    images_registered = len(rec.images)
                    best_idx = idx1
            except Exception:
                continue

    # Parse the reconstruction object to get the rotation matrix and translation vector
    # obtained for each image in the reconstruction
    results = {}
    camid_im_map = {}
    if best_idx is not None:
        for k, im in maps[best_idx].images.items():
            key = os.path.join(images_dir, im.name)
            results[key] = {}
            results[key]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
            results[key]["t"] = deepcopy(np.array(im.cam_from_world.translation))

            camid_im_map[im.camera_id] = im.name

    try:
        if best_idx is not None:
            for idx1, rec in maps.items():
                u_cameras = []
                g_cameras = []
                if idx1 != best_idx:
                    for k, im in rec.images.items():
                        key = os.path.join(images_dir, im.name)
                        if key in results:
                            g_R = deepcopy(results[key]["R"])
                            g_t = deepcopy(results[key]["t"])
                            g_C = -g_R.T @ g_t

                            u_R = deepcopy(im.cam_from_world.rotation.matrix())
                            u_t = deepcopy(np.array(im.cam_from_world.translation))
                            u_C = -u_R.T @ u_t
                            g_cameras.append(g_C.reshape(3, 1))
                            u_cameras.append(u_C.reshape(3, 1))
                    if len(g_cameras) < 3:
                        continue
                    g_cameras = np.array(g_cameras).reshape(3, -1)
                    u_cameras = np.array(u_cameras).reshape(3, -1)
                    inl_cf = 0
                    strict_cf = -1
                    thresholds = np.array([0.025, 0.05, 0.1, 0.2, 0.5, 1.0])
                    model = register_by_Horn(
                        u_cameras, g_cameras,
                        np.asarray(thresholds), inl_cf, strict_cf
                    )
                    T = np.squeeze(model["transf_matrix"][-1])
                    # print(T)
                    # print(T[:3].shape)
                    for k, im in rec.images.items():
                        key = os.path.join(images_dir, im.name)
                        if key not in results:
                            Tcw2 = np.eye(4)
                            Tcw2[:3, :3] = deepcopy(im.cam_from_world.rotation.matrix())
                            Tcw2[:3, 3] = deepcopy(np.array(im.cam_from_world.translation))
                            Tw2c = np.linalg.inv(Tcw2)
                            Tw1c = np.matmul(T, Tw2c)
                            Tcw1 = np.linalg.inv(Tw1c)
                            results[key]["R"] = deepcopy(Tcw1[:3, :3])
                            results[key]["t"] = deepcopy(Tcw1[:3, 3])
    except:
        pass
    # with open(os.path.join(work_dir, "camid_im_map.json"), "w") as f:
    #     json.dump(camid_im_map, f, indent=2)
    
    print(f"Registered: {dataset} / {scene} -> {len(results)} images")
    print(f"Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    # Create Submission
    submission = {
        "image_path": [],
        "dataset": [],
        "scene": [],
        "rotation_matrix": [],
        "translation_vector": []
    }
    for image in data_dict[dataset][scene]:
        if str(image) in results:
            print(image)
            R = results[str(image)]["R"].reshape(-1)
            T = results[str(image)]["t"].reshape(-1)
        else:
            R = np.eye(3).reshape(-1)
            T = np.zeros((3))
        image_path = str(image.relative_to(base_path))
        
        submission["image_path"].append(image_path)
        submission["dataset"].append(dataset)
        submission["scene"].append(scene)
        submission["rotation_matrix"].append(arr_to_str(R))
        submission["translation_vector"].append(arr_to_str(T))
    
    submission_df = pd.DataFrame.from_dict(submission)
    submission_path = os.path.join(work_dir, "submission.csv")
    print(f"Save to {submission_path}")
    submission_df.to_csv(submission_path, index=False)
    return submission_path


Overwriting imc2024-inference-scripts/reconstruction.py


In [15]:
%%writefile imc2024-inference-scripts/inference.py
import multiprocessing
import argparse
import os
import pandas as pd
import shutil
import json
import gc
import torch
from pathlib import Path
from config import Config
from pipeline import Pipeline
from reconstruction import reconstruction

def parse_sample_submission(
    base_path: str,
    input_csv: str,
    target_datasets: list,
) -> dict[dict[str, list[Path]]]:
    """Construct a dict describing the test data as 
    
    {"dataset": {"scene": [<image paths>]}}
    """
    data_dict = {}
    with open(input_csv, "r") as f:
        for i, l in enumerate(f):
            # Skip header
            if i == 0:
                print("header:", l)

            if l and i > 0:
                image_path, dataset, scene, _, _ = l.strip().split(',')
                if target_datasets is not None and dataset not in target_datasets:
                    continue

                ### Temp for bug?
                if not os.path.isfile(os.path.join(base_path,image_path)):
                    continue

                if dataset not in data_dict:
                    data_dict[dataset] = {}
                if scene not in data_dict[dataset]:
                    data_dict[dataset][scene] = []
                data_dict[dataset][scene].append(Path(os.path.join(base_path,image_path)))

    for dataset in data_dict:
        for scene in data_dict[dataset]:
            print(f"{dataset} / {scene} -> {len(data_dict[dataset][scene])} images")

    return data_dict

def worker_reconstruction(input_queue, submission_path_list):
    while True:
        reconstruction_inputs = input_queue.get()
        if reconstruction_inputs is None:
            break
        data_dict, dataset, scene, base_path, work_dir, colmap_mapper_options = reconstruction_inputs
        submission_path = reconstruction(data_dict, dataset, scene, base_path, work_dir, colmap_mapper_options)
        submission_path_list.append(submission_path)
        
def run(config):
    parser = argparse.ArgumentParser()
    parser.add_argument("--device_id", type=int, default=0)
    parser.add_argument("--target_datasets", nargs="*", required=False)
    parser.add_argument("--output_dir", required=False)
    args = parser.parse_args()

    if args.output_dir:
        config.output_dir = args.output_dir
    
    if args.target_datasets:
        config.target_datasets = args.target_datasets

    # Check output_dir
    if config.check_exist_dir:
        if os.path.isdir(config.output_dir):
            raise Exception(f"{config.output_dir} is already exists.")

    os.makedirs(config.output_dir, exist_ok=True)
    base_path = os.path.join(config.input_dir_root, "image-matching-challenge-2024")
    feature_dir = os.path.join(config.output_dir, "feature_outputs")
    shutil.copy(config.pipeline_json, config.output_dir)
    shutil.copy(config.transparent_pipeline_json, config.output_dir)

    # Load category
    category_df = pd.read_csv(config.category_csv)
    categories = {}
    for i, row in category_df.iterrows():
        categories[row["scene"]] = row["categories"].split(";")

    data_dict = parse_sample_submission(base_path, config.input_csv, config.target_datasets)
    datasets = list(data_dict.keys())
    
    manager = multiprocessing.Manager()
    submission_path_list = manager.list()

    input_queue = multiprocessing.Queue()
    worker_reconstruction_process = multiprocessing.Process(target=worker_reconstruction, args=(input_queue, submission_path_list))
    worker_reconstruction_process.start()
    for dataset in datasets:            
        for scene in data_dict[dataset]:
            print(f"[Scene] {scene}")
            work_dir = Path(os.path.join(feature_dir, f"{dataset}_{scene}"))
            work_dir.mkdir(parents=True, exist_ok=True)

            # Switching JSON
            if "transparent" in categories[scene]:
                json_path = config.transparent_pipeline_json
            else:
                json_path = config.pipeline_json
            print(f"catefories: {categories[scene]}")
            print(f"json path: {json_path}")
            
            # Exec Pipeline
            with open(json_path, "r") as f:
                pipeline_config = json.load(f)
            pipeline = Pipeline(data_dict[dataset][scene], work_dir, config.input_dir_root, pipeline_config, args.device_id)
            pipeline.exec()

            # Reconstruction & Save CSV
            print("Start Reconstruction")            
            torch.cuda.empty_cache()
            gc.collect()
            input_queue.put((data_dict, dataset, scene, base_path, work_dir, config.colmap_mapper_options))
    input_queue.put(None)
    worker_reconstruction_process.join()
    # Concat Submission
    if len(submission_path_list) > 0:
        submission_df_list = [pd.read_csv(p) for p in submission_path_list]
        submission_df = pd.concat(submission_df_list).reset_index(drop=True)
    else:
        submission_df = pd.DataFrame.from_dict({
            "image_path": [],
            "dataset": [],
            "scene": [],
            "rotation_matrix": [],
            "translation_vector": []
        })
    submission_df.to_csv(os.path.join(config.output_dir, "submission.csv"), index=False)

if __name__ == '__main__':
    cfg = Config
    run(cfg)

Overwriting imc2024-inference-scripts/inference.py


## Split scene list

In [16]:
root_path = "/kaggle/input/image-matching-challenge-2024/test"
data_num_list = [sum(len(files) for _, _, files in os.walk(f"{root_path}/{scene}/images")) for scene in non_transparent_scenes]
data_num_list

[41]

In [17]:
# Check correct path (if the path is wrong, it should be Notebook exeption)
if 0 in data_num_list:
    raise Exception

In [18]:
def partition_with_index(lst, lst2):
    indexed_lst = sorted(enumerate(lst), key=lambda x: x[1], reverse=True)
    groupA = []
    groupB = []
    groupA2 = []
    groupB2 = []
    
    for index, item in indexed_lst:
        if sum([lst[i] for i in groupA]) <= sum([lst[i] for i in groupB]):
            groupA.append(index)
            groupA2.append(lst2[index])
        else:
            groupB.append(index)
            groupB2.append(lst2[index])
    
    return groupA2, groupB2

In [19]:
non_transparent_scenes0, non_transparent_scenes1 = partition_with_index(data_num_list, non_transparent_scenes)
print(non_transparent_scenes0)
print(non_transparent_scenes1)

['church']
[]


## Exec Inference

In [20]:
torch.cuda.empty_cache()
import gc
gc.collect()

0

In [21]:
def make_command(device_id, target_datasets):
    cmd = f"python imc2024-inference-scripts/inference.py --device_id {device_id} --output_dir /tmp/tmp{device_id}"
    if len(target_datasets) > 0:
        cmd += " --target_datasets"
        for scene in target_datasets:
            cmd += f" {scene}"
    return cmd.split(" ")

In [22]:
import subprocess
proc0 = subprocess.Popen(make_command(0, non_transparent_scenes0))
proc1 = subprocess.Popen(make_command(1, non_transparent_scenes1))
proc0.wait()
proc1.wait()

2024-06-13 11:53:10.954533: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 11:53:10.954552: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 11:53:10.954628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 11:53:10.954642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 11:53:11.054716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory

DKM is not installed.Please install it if you want to use it.
header: image_path,dataset,scene,rotation_matrix,translation_vector

DKM is not installed.Please install it if you want to use it.
header: image_path,dataset,scene,rotation_matrix,translation_vector

church / church -> 41 images
[Scene] church
catefories: ['symmetries-and-repeats']
json path: /kaggle/input/configs-imc2024/1280_2048_crop_1280_2048/pipeline.json
device: cuda:0
===== [get_exif]  =====
===== [get_image_pair_exhaustive]  =====
image_num = 41
pair_num = 820
save -> /tmp/tmp0/feature_outputs/church_church/image_pair.csv
===== [rotate_matching_find_best] aliked_LightGlue (imsize=840) =====


Computing keypoints: 100%|██████████| 161/161 [00:08<00:00, 19.35it/s]


Matching 0 - 0
pair_num -> 820
Loaded LightGlue model


Computing keypoing distances: 100%|██████████| 820/820 [00:07<00:00, 115.20it/s]


...

I20240613 12:22:51.147107   128 misc.cc:205] 
Bundle adjustment report
------------------------
I20240613 12:22:51.147189   128 bundle_adjustment.cc:942] 
    Residuals : 1138572
   Parameters : 289006
   Iterations : 11
         Time : 16.2968 [s]
 Initial cost : 0.632677 [px]
   Final cost : 0.632007 [px]
  Termination : Convergence

I20240613 12:22:51.718345   128 incremental_mapper.cc:175] => Completed observations: 55
I20240613 12:22:51.946676   128 incremental_mapper.cc:178] => Merged observations: 203
I20240613 12:22:52.077651   128 incremental_mapper.cc:160] => Filtered observations: 21
I20240613 12:22:52.077716   128 incremental_mapper.cc:119] => Changed observations: 0.000490
I20240613 12:22:52.080327   128 incremental_mapper.cc:167] => Filtered images: 3
I20240613 12:22:52.100157   128 misc.cc:198] 
Registering image #39 (38)
I20240613 12:22:52.100190   128 incremental_mapper.cc:495] => Image sees 4668 / 5363 points
I20240613 12:22:53.235565   128 misc.cc:205] 
Pose refineme

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  4.580746e+05    0.00e+00    1.08e+05   0.00e+00   0.00e+00  1.00e+04        0    5.10e-01    2.84e+00
   1  4.575557e+05    5.19e+02    5.65e+02   4.20e+01   1.00e+00  3.00e+04        1    1.38e+00    4.23e+00
   2  4.575545e+05    1.22e+00    4.51e+03   2.36e+01   9.36e-01  8.88e+04        1    1.28e+00    5.51e+00
   3  4.575578e+05   -3.30e+00    4.51e+03   5.33e+01  -1.83e+00  4.44e+04        1    8.09e-01    6.32e+00
   4  4.575538e+05    6.91e-01    8.73e+03   2.93e+01   6.57e-01  4.58e+04        1    1.26e+00    7.58e+00
   5  4.575530e+05    8.07e-01    8.89e+03   2.66e+01   6.79e-01  4.81e+04        1    1.36e+00    8.94e+00
   6  4.575522e+05    7.31e-01    9.29e+03   2.47e+01   6.37e-01  4.91e+04        1    1.34e+00    1.03e+01
   7  4.575515e+05    7.15e-01    9.21e+03   2.25e+01   6.36e-01  5.01e+04        1    1.28e+00    1.16e+01
   8  4.575508e+05    6.66e-

I20240613 12:24:22.672819   128 misc.cc:205] 
Bundle adjustment report
------------------------
I20240613 12:24:22.672967   128 bundle_adjustment.cc:942] 
    Residuals : 1139982
   Parameters : 289150
   Iterations : 51
         Time : 69.7626 [s]
 Initial cost : 0.633898 [px]
   Final cost : 0.633527 [px]
  Termination : No convergence

I20240613 12:24:23.240712   128 incremental_mapper.cc:175] => Completed observations: 43
I20240613 12:24:23.462941   128 incremental_mapper.cc:178] => Merged observations: 64
I20240613 12:24:23.578485   128 incremental_mapper.cc:160] => Filtered observations: 18
I20240613 12:24:23.578547   128 incremental_mapper.cc:119] => Changed observations: 0.000219
I20240613 12:24:23.579212   128 incremental_mapper.cc:167] => Filtered images: 2
I20240613 12:24:23.622184   128 misc.cc:198] 
Finding good initial image pair
I20240613 12:24:23.627048   128 incremental_mapper.cc:404] => No good initial image pair found.
I20240613 12:24:23.627249   128 timer.cc:91] Ela

{0: Reconstruction(num_reg_images=38, num_cameras=38, num_points3D=96268, num_observations=569627)}
Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 38
	num_cameras = 38
	num_points3D = 96268
	num_observations = 569627
	mean_track_length = 5.9171
	mean_observations_per_image = 14990.2
	mean_reprojection_error = 0.766641
Registered: church / church -> 38 images
Total: church / church -> 41 images
/kaggle/input/image-matching-challenge-2024/test/church/images/00046.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00090.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00092.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00087.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00050.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00068.png
/kaggle/input/image-matching-challenge-2024/test/church/images/00083.png
/kaggle/input/image-matching-challenge-2024/test/church/images/0009

0

In [23]:
df1 = pd.read_csv("/tmp/tmp0/submission.csv")
df2 = pd.read_csv("/tmp/tmp1/submission.csv")
df = pd.concat([df1, df2]).reset_index(drop=True)
df.to_csv("/tmp/submission.csv", index=False)

## Merge csv

In [24]:
df1 = pd.read_csv(f"{OUTPUT_ROOT}/transp_submission.csv")
df2 = pd.read_csv("/tmp/submission.csv")

df1 = df1.query("scene not in @non_transparent_scenes").reset_index(drop=True)
df = pd.concat([df1, df2])

FINAL_SUB_TMP_PATH = "/tmp/final_submission.csv"
df.to_csv(FINAL_SUB_TMP_PATH, index=False)
print(df)

                      image_path dataset   scene  \
0   test/church/images/00046.png  church  church   
1   test/church/images/00090.png  church  church   
2   test/church/images/00092.png  church  church   
3   test/church/images/00087.png  church  church   
4   test/church/images/00050.png  church  church   
5   test/church/images/00068.png  church  church   
6   test/church/images/00083.png  church  church   
7   test/church/images/00096.png  church  church   
8   test/church/images/00069.png  church  church   
9   test/church/images/00081.png  church  church   
10  test/church/images/00042.png  church  church   
11  test/church/images/00018.png  church  church   
12  test/church/images/00030.png  church  church   
13  test/church/images/00024.png  church  church   
14  test/church/images/00032.png  church  church   
15  test/church/images/00026.png  church  church   
16  test/church/images/00037.png  church  church   
17  test/church/images/00008.png  church  church   
18  test/chu

In [25]:
%%writefile imc2024-inference-scripts/config.py

import os
import json

class Config:
    input_dir_root = "/kaggle/input"
    output_dir = "/tmp"
    check_exist_dir = False
    
    input_csv = "/kaggle/input/image-matching-challenge-2024/sample_submission.csv"
    category_csv = "/kaggle/input/image-matching-challenge-2024/test/categories.csv"
    target_datasets = []
    
    # matches >= 70
    pipeline_json = "/kaggle/input/imc2024-pipelines-v7/exp8_aliked1536+1280_crop-aliked1024_v2_100_70/pipeline.json"
    transparent_pipeline_json = "/kaggle/input/imc2024-pipelines-v7/exp8_aliked1536+1280_crop-aliked1024_v2_100_70/transp_pipeline.json"
    
    colmap_mapper_options = {
        "min_model_size": 3, # By default colmap does not generate a reconstruction if less than 10 images are registered. Lower it to 3.
        "max_num_models": 2,
        #"num_threads": 1,
    }

Overwriting imc2024-inference-scripts/config.py


In [26]:
thr_null_img = 0.15

In [27]:
# find scenes with low N of reg images
df1_ = pd.read_csv(FINAL_SUB_TMP_PATH)
df_non_trsp = df1_.query("scene in @non_transparent_scenes").reset_index(drop=True)
print(df_non_trsp.shape)

rot_m_null = '1.0;0.0;0.0;0.0;1.0;0.0;0.0;0.0;1.0'
tr_v_null = '0.0;0.0;0.0'

scene_cnt_dict = df_non_trsp.groupby(['scene']).image_path.count().to_dict()
print('scene_cnt_dict', scene_cnt_dict)

mask_null = (df_non_trsp['rotation_matrix'] == rot_m_null) & (df_non_trsp['translation_vector']==tr_v_null)
scene_cnt_null_dict = df_non_trsp[mask_null].groupby(['scene']).image_path.count().to_dict()

poor_non_trsp_scenes = [k_scene for k_scene, v_cnt in scene_cnt_null_dict.items() if v_cnt/scene_cnt_dict[k_scene] >= thr_null_img]
print('poor_non_trsp_scenes', poor_non_trsp_scenes)


(41, 5)
scene_cnt_dict {'church': 41}
poor_non_trsp_scenes []


In [28]:
data_num_list = [sum(len(files) for _, _, files in os.walk(f"{root_path}/{scene}/images")) for scene in poor_non_trsp_scenes]
data_num_list

[]

In [29]:
non_transparent_scenes0, non_transparent_scenes1 = partition_with_index(data_num_list, poor_non_trsp_scenes)
print(non_transparent_scenes0)
print(non_transparent_scenes1)

[]
[]


In [30]:
!rm -rf /tmp/tmp0 /tmp/tmp1

In [31]:
import gc
torch.cuda.empty_cache()
gc.collect()

12

In [32]:
import subprocess
proc0 = subprocess.Popen(make_command(0, non_transparent_scenes0))
proc1 = subprocess.Popen(make_command(1, non_transparent_scenes1))
proc0.wait()
proc1.wait()

2024-06-13 12:24:37.284497: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 12:24:37.284559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 12:24:37.286332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 12:24:37.435765: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 12:24:37.435811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factor

DKM is not installed.Please install it if you want to use it.
header: image_path,dataset,scene,rotation_matrix,translation_vector

DKM is not installed.Please install it if you want to use it.
header: image_path,dataset,scene,rotation_matrix,translation_vector



0

In [33]:
df1_ = pd.read_csv("/tmp/tmp0/submission.csv")
df2_ = pd.read_csv("/tmp/tmp1/submission.csv")
df_poor = pd.concat([df1_, df2_]).reset_index(drop=True)
df_poor.to_csv("/tmp/submission_poor.csv", index=False)

In [34]:
df_poor = pd.read_csv("/tmp/submission_poor.csv")

# use only those reconstructions which perform better e.g. more registered images
mask_null = (df_non_trsp['rotation_matrix']==rot_m_null) & (df_non_trsp['translation_vector']==tr_v_null)
scene_cnt_reg_dict = df_non_trsp[~mask_null].groupby(['scene']).image_path.count().to_dict()

mask_poor_null = (df_poor['rotation_matrix']==rot_m_null) & (df_poor['translation_vector']==tr_v_null)
scene_poor_cnt_reg_dict = df_poor[~mask_poor_null].groupby(['scene']).image_path.count().to_dict()

replace_rec_scenes = [k_scene for k_scene,v_reg_cnt in scene_poor_cnt_reg_dict.items() if v_reg_cnt > scene_cnt_reg_dict[k_scene]]
print('Update reconstructions with', replace_rec_scenes)

df_non_trsp_new = df_non_trsp[~df_non_trsp.scene.isin(replace_rec_scenes)]
df_poor_new = df_poor[df_poor.scene.isin(replace_rec_scenes)]

# updated non transparent scenes
df_non_trsp_new = df_non_trsp_new.reset_index(drop=True)
df_poor_new = df_poor_new.reset_index(drop=True)
df_non_trnsp_ = pd.concat([df_non_trsp_new, df_poor_new]).reset_index(drop=True)
df_non_trnsp_.to_csv("/tmp/submission.csv", index=False)

Update reconstructions with []


In [35]:
scene_poor_cnt_reg_dict, scene_cnt_reg_dict

({}, {'church': 38})

In [36]:
if len(replace_rec_scenes) > 0:
    df1_ = pd.read_csv(f"{OUTPUT_ROOT}/transp_submission.csv") # transparent
    df1_ = df1_.query("scene not in @non_transparent_scenes").reset_index(drop=True)
    df2_ = pd.read_csv("/tmp/submission.csv") # non transparent
    
    df = pd.concat([df1_, df2_])
    df.to_csv(FINAL_SUB_TMP_PATH, index=False)
    print(df)

In [37]:
# copy tmp sub to final location (to avoid kaggle early stop)
FINAL_SUB_PATH = f"{OUTPUT_ROOT}/submission.csv"
!cp {FINAL_SUB_TMP_PATH} {FINAL_SUB_PATH}